In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import re
import statsmodels.api as sm
import statsmodels.formula.api as smf
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import cross_validation
from datetime import datetime, timedelta
import calendar
import holidays
from dateutil.relativedelta import relativedelta

%matplotlib inline

In [3]:
#viz setup
# sns.set(style='whitegrid',font_scale=1.75,rc={"axes.spines.top":False,"axes.spines.right":False, "lines.linewidth": 2.5,'lines.markersize': 10},color_codes=False,palette=sns.color_palette(['#27a3aa','#f76d23','#70d6e3','#ffbb31','#b1c96d','#cce18a','#1c4c5d','#787642']))
sns.set(style='whitegrid',font_scale=1.5,rc={"axes.spines.top":False,"axes.spines.right":False, "lines.linewidth": 2.5,'lines.markersize': 10},color_codes=False,palette=sns.color_palette(['#27a3aa','#f76d23','#70d6e3','#ffbb31','#b1c96d','#cce18a','#1c4c5d','#787642']))

In [4]:
states = ["AL", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
regions = ["_ENC","_MAC","_MTN","_NEC","_PAC","PUS","_WNC","_WSC","_ESC","_SAC"]
sectors = ['RES','COM']

In [5]:
idx = 0
for state in states:
    for sector in sectors:
        print("{}-{}".format(state,sector))
        response_consumption = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=ELEC.SALES.{}-{}.M".format(state,sector))
        j_consumption = response_consumption.json()
        tmp_consumption = pd.DataFrame(j_consumption['series'][0]['data'],columns=['month','sales_mkwh'])
        tmp_consumption['state'] = state
        tmp_consumption['sector'] = sector
        
        response_consumers = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=ELEC.CUSTOMERS.{}-{}.M".format(state,sector))
        j_consumers = response_consumers.json()
        tmp_consumers = pd.DataFrame(j_consumers['series'][0]['data'],columns=['month','consumers'])
        tmp_consumers['state'] = state
        tmp_consumers['sector'] = sector
        
        response_price = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=ELEC.PRICE.{}-{}.M".format(state,sector))
        j_price = response_price.json()
        tmp_price = pd.DataFrame(j_price['series'][0]['data'],columns=['month','price'])
        tmp_price['state'] = state
        tmp_price['sector'] = sector
        
        tmp = tmp_consumption.merge(tmp_consumers,how='left',on=['month','state','sector']).merge(tmp_price,how='left',on=['month','state','sector'])
        
        if idx == 0:
            energy_data = tmp.copy()
        else:
            energy_data = energy_data.append(tmp)
        idx = idx +1

AL-RES
AL-COM
AZ-RES
AZ-COM
AR-RES
AR-COM
CA-RES
CA-COM
CO-RES
CO-COM
CT-RES
CT-COM
DC-RES
DC-COM
DE-RES
DE-COM
FL-RES
FL-COM
GA-RES
GA-COM
ID-RES
ID-COM
IL-RES
IL-COM
IN-RES
IN-COM
IA-RES
IA-COM
KS-RES
KS-COM
KY-RES
KY-COM
LA-RES
LA-COM
ME-RES
ME-COM
MD-RES
MD-COM
MA-RES
MA-COM
MI-RES
MI-COM
MN-RES
MN-COM
MS-RES
MS-COM
MO-RES
MO-COM
MT-RES
MT-COM
NE-RES
NE-COM
NV-RES
NV-COM
NH-RES
NH-COM
NJ-RES
NJ-COM
NM-RES
NM-COM
NY-RES
NY-COM
NC-RES
NC-COM
ND-RES
ND-COM
OH-RES
OH-COM
OK-RES
OK-COM
OR-RES
OR-COM
PA-RES
PA-COM
RI-RES
RI-COM
SC-RES
SC-COM
SD-RES
SD-COM
TN-RES
TN-COM
TX-RES
TX-COM
UT-RES
UT-COM
VT-RES
VT-COM
VA-RES
VA-COM
WA-RES
WA-COM
WV-RES
WV-COM
WI-RES
WI-COM
WY-RES
WY-COM


In [6]:
idx = 0
for region in regions:
    response_cool = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=STEO.ZWCD{}.M".format(region))
    j_cool = response_cool.json()
    tmp_cool = pd.DataFrame(j_cool['series'][0]['data'],columns=['month','cooling_days'])
    tmp_cool['region'] = region
    
    response_heat = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=STEO.ZWHD{}.M".format(region))
    j_heat = response_heat.json()
    tmp_heat = pd.DataFrame(j_heat['series'][0]['data'],columns=['month','heating_days'])
    tmp_heat['region'] = region
    
    tmp = tmp_cool.merge(tmp_heat,how='left',on=['month','region'])
    if idx == 0:
        heating_cooling_days = tmp.copy()
    else:
        heating_cooling_days = heating_cooling_days.append(tmp)
    idx = idx +1

In [7]:
energy_data['revenue'] = energy_data.sales_mkwh*energy_data.price
country = energy_data.groupby(['month','sector']).sum().reset_index()
country['state'] = 'USA'
country.price = country.revenue/country.sales_mkwh

In [8]:
energy_data = energy_data.append(country)

In [9]:
energy_data['use_per_capita'] = energy_data.sales_mkwh*1000000/energy_data.consumers

In [10]:
heating_cooling_days.region = [re.sub('_','',r) for r in heating_cooling_days.region]

In [11]:
states.extend(['USA'])

In [12]:
state_region_mapping = pd.DataFrame(data={'state': states})

In [13]:
state_region_mapping['region'] = ''

In [14]:
state_region_mapping.loc[state_region_mapping.state.isin(['WA','OR','CA']),'region'] = 'PAC'
state_region_mapping.loc[state_region_mapping.state.isin(['MT','ID','WY','NV','UT','CO','AZ','NM']),'region'] = 'MTN'
state_region_mapping.loc[state_region_mapping.state.isin(['ND','SD','MN','NE','IA','KS','MO']),'region'] = 'WNC'
state_region_mapping.loc[state_region_mapping.state.isin(['OK','TX','AR','LA']),'region'] = 'WSC'
state_region_mapping.loc[state_region_mapping.state.isin(['WI','IL','IN','MI','OH']),'region'] = 'ENC'
state_region_mapping.loc[state_region_mapping.state.isin(['KY','TN','MS','AL']),'region'] = 'ESC'
state_region_mapping.loc[state_region_mapping.state.isin(['WV','MD','DE','VA','NC','SC','GA','FL','DC']),'region'] = 'SAC'
state_region_mapping.loc[state_region_mapping.state.isin(['NY','PA','NJ']),'region'] = 'MAC'
state_region_mapping.loc[state_region_mapping.state.isin(['RI','CT','MA','NH','VT','ME']),'region'] = 'NEC'
state_region_mapping.loc[state_region_mapping.state.isin(['USA']),'region'] = 'PUS'

In [15]:
energy_data = energy_data.merge(state_region_mapping,how='left',on='state')

In [16]:
energy_data= energy_data.merge(heating_cooling_days,how='left',on=['month','region'])

In [17]:
energy_data = energy_data.dropna()

In [18]:
energy_data = pd.concat([energy_data,pd.get_dummies(energy_data.sector)],axis=1)

In [19]:
energy_data['time'] = [12*(int(d[0:4])-2008)+int(d[4:6]) for d in energy_data.month]

In [20]:
energy_data['year'] = [int(d[0:4]) for d in energy_data.month]
energy_data['mon'] = [int(d[4:6]) for d in energy_data.month]

In [21]:
def get_season(m):
    if (m == 12)|(m<=2):
        return 'winter'
    if (m>=3)&(m<=5):
        return 'spring'
    if(m>=6)&(m<=8):
        return 'summer'
    if(m>=9)&(m<=11):
        return 'fall'

In [22]:
energy_data['season'] = energy_data.mon.apply(get_season)

In [23]:
energy_data.head()

,month,sales_mkwh,state,sector,consumers,price,revenue,use_per_capita,region,cooling_days,heating_days,COM,RES,time,year,mon,season
0,202105,2191.98207,AL,RES,2299125.0,13.54,29679.437228,953.398389,ESC,109.419607,81.650518,0,1,161,2021,5,spring
1,202104,1978.04896,AL,RES,2294478.0,13.73,27158.612221,862.091055,ESC,17.744106,232.581562,0,1,160,2021,4,spring
2,202103,2321.97408,AL,RES,2295326.0,13.35,30998.353968,1011.609715,ESC,33.840873,339.104453,0,1,159,2021,3,spring
3,202102,2906.25268,AL,RES,2247306.0,12.99,37752.222313,1293.216269,ESC,0.817566,718.891759,0,1,158,2021,2,winter
4,202101,3164.80449,AL,RES,2284207.0,12.38,39180.279586,1385.515625,ESC,5.622370,736.642738,0,1,157,2021,1,winter


In [24]:
energy_data['date'] = [datetime(y,m,1) for y,m in zip(energy_data.year, energy_data.mon)]

In [25]:
def get_datetime_features(date):
    st = date
    en = date + relativedelta(months=1) - relativedelta(days=1)
    
    ## number of days in month
    num_days = len(pd.date_range(st,en))
    ## number of weekends in month
    num_weekends = pd.date_range(st,en).weekday.isin([5,6]).sum()
    ## number of holidays in month
    us_holidays = holidays.US(years=date.year)
    us_holidays = pd.DataFrame(us_holidays.items(),columns=['date','hol'])
    us_holidays['date'] = pd.to_datetime(us_holidays.date)
    num_holidays = len(us_holidays[(us_holidays.date.dt.month == date.month) & ~(us_holidays.date.dt.weekday.isin([5,6]))])
    
    num_weekends_or_holidays = num_holidays+num_weekends
    ## % of weekdays in month
    pct_weekdays = 1 - (num_holidays+num_weekends)/num_days
    
    return num_days, num_weekends_or_holidays, pct_weekdays

In [26]:
energy_data['num_days'], energy_data['num_hols'], energy_data['pct_weekdays'] = zip(*energy_data.date.apply(get_datetime_features)) 

In [27]:
energy_data['y'] = energy_data.use_per_capita/energy_data.num_days

In [28]:
energy_data.head()

,month,sales_mkwh,state,sector,consumers,price,revenue,use_per_capita,region,cooling_days,...,RES,time,year,mon,season,date,num_days,num_hols,pct_weekdays,y
0,202105,2191.98207,AL,RES,2299125.0,13.54,29679.437228,953.398389,ESC,109.419607,...,1,161,2021,5,spring,2021-05-01,31,11,0.645161,30.754787
1,202104,1978.04896,AL,RES,2294478.0,13.73,27158.612221,862.091055,ESC,17.744106,...,1,160,2021,4,spring,2021-04-01,30,8,0.733333,28.736369
2,202103,2321.97408,AL,RES,2295326.0,13.35,30998.353968,1011.609715,ESC,33.840873,...,1,159,2021,3,spring,2021-03-01,31,8,0.741935,32.632571
3,202102,2906.25268,AL,RES,2247306.0,12.99,37752.222313,1293.216269,ESC,0.817566,...,1,158,2021,2,winter,2021-02-01,28,9,0.678571,46.186295
4,202101,3164.80449,AL,RES,2284207.0,12.38,39180.279586,1385.515625,ESC,5.622370,...,1,157,2021,1,winter,2021-01-01,31,12,0.612903,44.694052


In [29]:
energy_data.tail()

,month,sales_mkwh,state,sector,consumers,price,revenue,use_per_capita,region,cooling_days,...,RES,time,year,mon,season,date,num_days,num_hols,pct_weekdays,y
24495,202103,114177.60253,USA,RES,137727290.0,13.239410,1.511644e+06,829.012192,PUS,28.144002,...,1,159,2021,3,spring,2021-03-01,31,8,0.741935,26.742329
24496,202104,97874.63383,USA,COM,18871777.0,10.937048,1.070460e+06,5186.296650,PUS,36.803609,...,0,160,2021,4,spring,2021-04-01,30,8,0.733333,172.876555
24497,202104,93833.56430,USA,RES,136903106.0,13.705571,1.286043e+06,685.401282,PUS,36.803609,...,1,160,2021,4,spring,2021-04-01,30,8,0.733333,22.846709
24498,202105,103992.29422,USA,COM,18861146.0,10.774514,1.120466e+06,5513.572411,PUS,102.374190,...,0,161,2021,5,spring,2021-05-01,31,11,0.645161,177.857175
24499,202105,101114.95221,USA,RES,136717815.0,13.652533,1.380475e+06,739.588708,PUS,102.374190,...,1,161,2021,5,spring,2021-05-01,31,11,0.645161,23.857700


In [30]:
energy_data.date.max()

Timestamp('2021-05-01 00:00:00')

In [31]:
data_urls = ['https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2008_c20180718.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2009_c20180718.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2010_c20200922.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2011_c20180718.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2012_c20200317.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2013_c20170519.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2014_c20210120.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2015_c20191116.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2016_c20190817.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2017_c20210120.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2018_c20210716.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2019_c20210604.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2020_c20210716.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2021_c20210716.csv.gz']

In [32]:
idx=0
for d in data_urls:
    print(d)
    tmp = pd.read_csv(d)
    if idx == 0:
        storm_data = tmp.copy()
    else:
        storm_data = storm_data.append(tmp)
    idx = idx +1

https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2008_c20180718.csv.gz
https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2009_c20180718.csv.gz
https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2010_c20200922.csv.gz
https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2011_c20180718.csv.gz
https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2012_c20200317.csv.gz
https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2013_c20170519.csv.gz
https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2014_c20210120.csv.gz
https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2015_c20191116.csv.gz
https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1

In [33]:
storm_data.EVENT_TYPE.value_counts()

Thunderstorm Wind             212533
Hail                          147734
Winter Weather                 51266
Flash Flood                    50308
Winter Storm                   43040
High Wind                      42964
Drought                        38073
Flood                          35799
Heavy Snow                     31647
Marine Thunderstorm Wind       23904
Tornado                        18464
Heavy Rain                     17964
Strong Wind                    14605
Heat                           10576
Extreme Cold/Wind Chill         9756
Blizzard                        9164
Frost/Freeze                    9096
Dense Fog                       9072
Excessive Heat                  7911
Lightning                       6894
Cold/Wind Chill                 6355
High Surf                       6141
Funnel Cloud                    4736
Wildfire                        4700
Ice Storm                       4158
Tropical Storm                  3442
Waterspout                      2644
C

In [34]:
storm_data[storm_data.DAMAGE_PROPERTY == '629.00M']

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
34007,201608,12,415,201608,13,630,109514,656655,LOUISIANA,22,...,2.0,WSW,BAPTIST,30.999,-90.566,30.487,-90.567,A slow moving low pressure system brought wide...,Twelve to eighteen inches of rainfall over a 2...,CSV


In [35]:
# storm_data_clean = storm_data[storm_data.MAGNITUDE>60].copy()
storm_data_clean = storm_data[['BEGIN_YEARMONTH', 'BEGIN_DAY', 'END_YEARMONTH',
       'END_DAY', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'EVENT_TYPE','MAGNITUDE', 'CATEGORY', 'TOR_F_SCALE',
       'EPISODE_NARRATIVE']].copy()

In [36]:
storm_data_clean = storm_data_clean.drop_duplicates(subset=['EVENT_TYPE','EPISODE_ID','STATE'])

In [37]:
storm_data_clean.tail()

,BEGIN_YEARMONTH,BEGIN_DAY,END_YEARMONTH,END_DAY,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,EVENT_TYPE,MAGNITUDE,CATEGORY,TOR_F_SCALE,EPISODE_NARRATIVE
19155,202103,13,202103,13,155514,937970,NEW YORK,36,High Wind,36.0,NaN,NaN,A strong cold front passed through the region ...
19158,202103,14,202103,14,155515,937973,NEW YORK,36,High Wind,50.0,NaN,NaN,A strong cold front passed through the region ...
19178,202103,24,202103,24,156695,946959,TEXAS,48,Funnel Cloud,NaN,NaN,NaN,A dryline out ahead of an approaching upper le...
19189,202103,18,202103,18,156988,948939,VIRGINIA,51,Tornado,NaN,NaN,EF0,Scattered severe thunderstorms in advance of l...
19195,202103,11,202103,15,157005,951084,MISSOURI,29,Heavy Rain,NaN,NaN,NaN,Heavy rainfall affected the region from the la...


In [38]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'USA':'USA'
}

In [39]:
us_state_abbrev_caps = {k.upper():v.upper() for k,v in us_state_abbrev.items()}

In [40]:
storm_data_clean['state'] = storm_data_clean.STATE.map(us_state_abbrev_caps)

In [41]:
storm_data_clean = storm_data_clean[~storm_data_clean.state.isna()]
storm_data_clean = storm_data_clean.drop(columns='STATE')

In [42]:
storm_data_clean['begin_date'] = [str(y) + str(d).zfill(2) for y,d in zip(storm_data_clean.BEGIN_YEARMONTH,storm_data_clean.BEGIN_DAY)]
storm_data_clean['end_date'] = [str(y) + str(d).zfill(2) for y,d in zip(storm_data_clean.END_YEARMONTH,storm_data_clean.END_DAY)]

In [43]:
storm_data_clean.begin_date = pd.to_datetime(storm_data_clean.begin_date)
storm_data_clean.end_date = pd.to_datetime(storm_data_clean.end_date)

In [44]:
storm_data_clean = storm_data_clean.drop_duplicates(subset='EPISODE_NARRATIVE').sort_values(['state','begin_date'])

In [45]:
storm_data_clean['num_days'] = (storm_data_clean.end_date - storm_data_clean.begin_date)

In [46]:
storm_data_clean.num_days = storm_data_clean.num_days.dt.days +1

In [47]:
storm_data_clean = storm_data_clean.drop(columns=['BEGIN_DAY','END_YEARMONTH','END_DAY'])

In [48]:
# events_to_keep = ['Thunderstorm Wind', 'Hail', 'Flash Flood', 'Flood', 'High Wind',
#        'Winter Weather', 'Tornado', 'Winter Storm', 'Heavy Snow', 'Heavy Rain',
#        'Lightning', 'Strong Wind', 'Blizzard', 'Heat', 'Frost/Freeze',
#        'Extreme Cold/Wind Chill', 'Excessive Heat', 'Cold/Wind Chill',
#        'Lake-Effect Snow',
#        'Ice Storm','Tropical Storm', 'Freezing Fog', 
#        'Hurricane (Typhoon)', 
#        'Hurricane']
events_to_keep = [
       'Winter Weather', 'Winter Storm', 'Heavy Snow', 'Blizzard', 'Heat', 'Frost/Freeze',
       'Extreme Cold/Wind Chill', 'Excessive Heat', 'Cold/Wind Chill','Lake-Effect Snow','Ice Storm',
        'Thunderstorm Wind', 'High Wind','Tornado','Heavy Rain','Strong Wind','Tropical Storm', 'Hurricane (Typhoon)', 'Hurricane']
hot_cold_map = {'Winter Weather':'cold', 'Winter Storm':'cold', 'Heavy Snow':'cold', 'Blizzard':'cold', 'Heat':'hot', 'Frost/Freeze':'cold',
       'Extreme Cold/Wind Chill':'cold', 'Excessive Heat':'hot', 'Cold/Wind Chill':'cold','Lake-Effect Snow':'cold','Ice Storm':'cold',
               'Thunderstorm Wind':'wind', 'High Wind':'wind','Tornado':'wind','Heavy Rain':'wind','Strong Wind':'wind','Tropical Storm':'wind',
                'Hurricane (Typhoon)':'wind', 'Hurricane':'wind'}

In [49]:
storm_data_clean = storm_data_clean[storm_data_clean.EVENT_TYPE.isin(events_to_keep)]
storm_data_clean['hot_cold'] = storm_data_clean.EVENT_TYPE.map(hot_cold_map)

In [50]:
storm_data_clean = storm_data_clean.groupby(['state','BEGIN_YEARMONTH','hot_cold']).sum().reset_index()

In [51]:
storm_data_clean = storm_data_clean.rename(columns={'BEGIN_YEARMONTH':'month'})
storm_data_clean.month = storm_data_clean.month.astype(str)

In [52]:
storm_data_clean = storm_data_clean.pivot(index=['state','month'], columns='hot_cold', values='num_days').reset_index().fillna(0)

In [53]:
storm_data_clean['mon'] = [x[4:6] for x in storm_data_clean.month]

In [54]:
storm_data_clean

hot_cold,state,month,cold,hot,wind,mon
0,AK,200801,26.0,0.0,7.0,01
1,AK,200802,42.0,0.0,7.0,02
2,AK,200803,42.0,0.0,6.0,03
3,AK,200804,19.0,0.0,3.0,04
4,AK,200805,0.0,0.0,3.0,05
...,...,...,...,...,...,...
7776,WY,202012,11.0,0.0,15.0,12
7777,WY,202101,16.0,0.0,23.0,01
7778,WY,202102,38.0,0.0,13.0,02
7779,WY,202103,10.0,0.0,9.0,03


In [55]:
avg_may = storm_data_clean.groupby(['state','mon']).mean().reset_index()
avg_may = avg_may[avg_may.mon.isin(['05'])]
avg_may['month'] = '2021'
avg_may['month'] = [x+y for x,y in zip(avg_may.month, avg_may.mon)]

In [56]:
avg_may

hot_cold,state,mon,cold,hot,wind,month
4,AK,05,1.888889,0.000000,2.111111,202105
16,AL,05,0.000000,0.153846,6.307692,202105
28,AR,05,0.076923,0.000000,10.384615,202105
40,AS,05,0.000000,0.000000,3.250000,202105
52,AZ,05,0.916667,1.666667,3.583333,202105
64,CA,05,4.076923,1.461538,8.923077,202105
76,CO,05,9.769231,0.000000,4.846154,202105
88,CT,05,0.400000,0.000000,2.100000,202105
100,DC,05,0.000000,0.000000,1.000000,202105
111,DE,05,0.000000,0.000000,1.125000,202105


In [57]:
storm_data_clean = pd.concat([storm_data_clean, avg_may])

In [58]:
storm_data_clean.month.value_counts()

202105    55
201001    53
201212    52
201801    52
201602    52
          ..
200809    43
201711    41
201610    38
201211    38
200911    37
Name: month, Length: 161, dtype: int64

In [59]:
energy_data = energy_data.merge(storm_data_clean[['state','month','hot','cold','wind']],how='left',on=['state','month'])
# energy_data.storm_days = energy_data.storm_days.fillna(0)

In [60]:
energy_data.hot = energy_data.hot.fillna(0)
energy_data.cold = energy_data.cold.fillna(0)
energy_data.wind = energy_data.wind.fillna(0)

In [61]:
energy_data.tail()

,month,sales_mkwh,state,sector,consumers,price,revenue,use_per_capita,region,cooling_days,...,mon,season,date,num_days,num_hols,pct_weekdays,y,hot,cold,wind
16263,202103,114177.60253,USA,RES,137727290.0,13.239410,1.511644e+06,829.012192,PUS,28.144002,...,3,spring,2021-03-01,31,8,0.741935,26.742329,0.0,0.0,0.0
16264,202104,97874.63383,USA,COM,18871777.0,10.937048,1.070460e+06,5186.296650,PUS,36.803609,...,4,spring,2021-04-01,30,8,0.733333,172.876555,0.0,0.0,0.0
16265,202104,93833.56430,USA,RES,136903106.0,13.705571,1.286043e+06,685.401282,PUS,36.803609,...,4,spring,2021-04-01,30,8,0.733333,22.846709,0.0,0.0,0.0
16266,202105,103992.29422,USA,COM,18861146.0,10.774514,1.120466e+06,5513.572411,PUS,102.374190,...,5,spring,2021-05-01,31,11,0.645161,177.857175,0.0,0.0,0.0
16267,202105,101114.95221,USA,RES,136717815.0,13.652533,1.380475e+06,739.588708,PUS,102.374190,...,5,spring,2021-05-01,31,11,0.645161,23.857700,0.0,0.0,0.0


In [62]:
energy_data.to_csv('../data/energy_data.csv',index=False)